# Upgrad CPP price extract

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Input file path
input_file = "C:\\Users\\taslim.siddiqui\\Downloads\\upgrad.xlsx"
df = pd.read_excel(input_file)

# Make sure column exists
if 'Course link' not in df.columns:
    raise ValueError("Excel must contain a 'Course link' column.")

# Results list
total_prices = []

# Headers for the request
headers = {
    "User-Agent": "Mozilla/5.0"
}

# Process each URL
for index, row in df.iterrows():
    url = row['Course link']
    print(f"🔍 Scraping: {url}")
    
    total_price = "❌ Total not found"
    
    try:
        response = requests.get(url, headers=headers, timeout=15)
        soup = BeautifulSoup(response.content, "html.parser")

        # First attempt: <h3 class="ProgramPricingWithSpecialization_price__MSWen">
        h3_tag = soup.find("h3", class_="ProgramPricingWithSpecialization_price__MSWen")
        if h3_tag:
            total_price = h3_tag.get_text(strip=True).split(" ")[0] + " " + h3_tag.get_text(strip=True).split(" ")[1]
        else:
            # Second attempt: <span class="font-medium text-bodySmall text-greyscale-main ...">
            span_tag = soup.find("span", class_="font-medium text-bodySmall text-greyscale-main ml-spacing4 md:-tracking-0.16 xs:-tracking-0.14")
            if span_tag:
                total_price = span_tag.get_text(strip=True)
    
    except Exception as e:
        total_price = f"❌ Error: {str(e)}"
    
    total_prices.append(total_price)
    print(f"✅ Total Price: {total_price}")  # Print the output for each course

# Add results to DataFrame
df['Total_Price'] = total_prices

# Save to Excel
output_file = "C:\\Users\\taslim.siddiqui\\Downloads\\upgrad_courses_with_prices.xlsx"
df.to_excel(output_file, index=False)
print(f"\n✅ All done. Results saved to: {output_file}")
